## 2. Giải quyết vấn đề

In [7]:
import pandas as pd
import numpy as np

### 2.1. Dữ liệu nhà hàng

#### 2.1.1. File `air_visit_data.csv`

- **Input:** File air_visit_data thu thập những dữ liệu về khách hàng đến cửa hàng theo ngày (nếu ngày mà cửa hàng không có khách thì file sẽ không hiển thị)
- **Output:** Dữ liệu khách hàng của cửa hàng sẽ được tính tổng theo từng ngày
- **Phương pháp:** chúng ta sẽ gom nhóm theo ngày và tính tổng số lượng khách của cửa hàng theo ngày đó, nếu ngày nào không có khách thì sẽ gắn số khách là 0
- **Mục đích:** tạo một chuỗi thời gian liên tục cho mỗi cửa hàng, từ đó m hình có thể học được xu hướng và mô hình vắng khách hoặc đóng cửa của nhà hàng 

In [8]:
path1 = 'data/air_visit_data.csv'
air_visit = pd.read_csv(path1)
air_visit.head(10)

,air_store_id,visit_date,visitors
0,air_ba937bf13d40fb24,2016-01-13,25
1,air_ba937bf13d40fb24,2016-01-14,32
2,air_ba937bf13d40fb24,2016-01-15,29
3,air_ba937bf13d40fb24,2016-01-16,22
4,air_ba937bf13d40fb24,2016-01-18,6
5,air_ba937bf13d40fb24,2016-01-19,9
6,air_ba937bf13d40fb24,2016-01-20,31
7,air_ba937bf13d40fb24,2016-01-21,21
8,air_ba937bf13d40fb24,2016-01-22,18
9,air_ba937bf13d40fb24,2016-01-23,26


In [9]:
air_visit.index = pd.to_datetime(air_visit['visit_date'])
air_visit = air_visit.groupby('air_store_id').apply(lambda data: data['visitors'].resample('1d').sum(), include_groups=False).reset_index()
air_visit['visit_date'] = air_visit['visit_date'].dt.strftime('%Y-%m-%d') #chuyen dang YYYY-mm-dd
air_visit.replace(0, np.nan, inplace=True)
air_visit['was_nil'] = air_visit['visitors'].isnull()
air_visit.fillna({'visitors': 0}, inplace = True) #ignore

air_visit.head(10)

,air_store_id,visit_date,visitors,was_nil
0,air_00a91d42b08b08d9,2016-07-01,35.0,False
1,air_00a91d42b08b08d9,2016-07-02,9.0,False
2,air_00a91d42b08b08d9,2016-07-03,0.0,True
3,air_00a91d42b08b08d9,2016-07-04,20.0,False
4,air_00a91d42b08b08d9,2016-07-05,25.0,False
5,air_00a91d42b08b08d9,2016-07-06,29.0,False
6,air_00a91d42b08b08d9,2016-07-07,34.0,False
7,air_00a91d42b08b08d9,2016-07-08,42.0,False
8,air_00a91d42b08b08d9,2016-07-09,11.0,False
9,air_00a91d42b08b08d9,2016-07-10,0.0,True


##

#### 2.1.2 `File date_info.csv`

- **Input:** file cung cấp cho chúng ta về thứ ngày tháng năm, với nhãn holiday_flg nhắm đánh dấu ngày đó có phải là ngày lễ không
- **Output:** sẽ có thêm 2 cột là 'prev_holiday_day' và 'after_holiday_day'
- **Phương pháp:** chúng ta sẽ thêm 2 biến 'prev_holiday_day' và 'after_holiday_day' với giá trị 0 để đánh dấu là ngày thường và 1 là ngày lễ 
- **Mục đích:** để mô hình có thể học được rằng đó là ngày thường hay lễ và liệu có phải là lễ dài hay không

In [10]:
date_info = pd.read_csv('data/date_info.csv')
date_info.rename(columns = {'calendar_date' : 'visit_date', 'holiday_flg' : 'is_holiday'}, inplace = True)
date_info['prev_holiday'] = date_info['is_holiday'].shift().fillna(0)
date_info['after_holiday'] = date_info['is_holiday'].shift(-1).fillna(1)
date_info.head()

,visit_date,day_of_week,is_holiday,prev_holiday,after_holiday
0,2016-01-01,Friday,1,0.0,1.0
1,2016-01-02,Saturday,1,1.0,1.0
2,2016-01-03,Sunday,1,1.0,0.0
3,2016-01-04,Monday,0,1.0,0.0
4,2016-01-05,Tuesday,0,0.0,0.0


#### 2.1.3 `File air_store_info.csv`

- **Output:** sẽ cho chúng ta dữ liệu về cửa hàng với những dữ liệu như vị trí ...
- **Mục đích:** mô hình sẽ học về những thứ liên quan đến cửa hàng như vị trí ... để có thể thông qua thông tin của hàng để có thể đoán tốt hơn

In [11]:
path3 = 'weather/air_store_info_with_nearest_active_station.csv'
air_store_info = pd.read_csv(path3)
air_store_info.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude,latitude_str,longitude_str,station_id,station_latitude,station_longitude,station_vincenty,station_great_circle
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197853,"""34.6951242""","""135.1978525""",hyogo__kobe-kana__koube,34.696667,135.211667,1.277232,1.274882
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599,"""35.6580681""","""139.7515992""",tokyo__tokyo-kana__tonokyo,35.691667,139.750000,3.730672,3.739835


#### 2.1.4 `File submission.csv`

- **Input:** cho ta dữ liệu một tập test
- **Output:**  cho ta dữ liệu với cột 'air_store_id' sẽ là id của cửa hàng và 'visit_date' là ngày có khách
- **Phương pháp:** dùng str.slice để cắt id cửa hàng và ngày có khách từ cột id

In [12]:
submission = pd.read_csv('data/sample_submission.csv')
submission['air_store_id'] = submission['id'].str.slice(0,20)
submission['visit_date'] = submission['id'].str.slice(21)
submission['visit_date'] = pd.to_datetime(submission['visit_date']).dt.strftime('%Y-%m-%d')
submission['visitors'] = np.nan #để bỏ qua cột này
submission['is_test'] = True
submission['is_number'] = submission.index

submission.head()


,id,visitors,air_store_id,visit_date,is_test,is_number
0,air_00a91d42b08b08d9_2017-04-23,NaN,air_00a91d42b08b08d9,2017-04-23,True,0
1,air_00a91d42b08b08d9_2017-04-24,NaN,air_00a91d42b08b08d9,2017-04-24,True,1
2,air_00a91d42b08b08d9_2017-04-25,NaN,air_00a91d42b08b08d9,2017-04-25,True,2
3,air_00a91d42b08b08d9_2017-04-26,NaN,air_00a91d42b08b08d9,2017-04-26,True,3
4,air_00a91d42b08b08d9_2017-04-27,NaN,air_00a91d42b08b08d9,2017-04-27,True,4


In [13]:
#xóa cột id
submission = submission.drop(['id'], axis = 'columns')
print(submission.head())

   visitors          air_store_id  visit_date  is_test  is_number
0       NaN  air_00a91d42b08b08d9  2017-04-23     True          0
1       NaN  air_00a91d42b08b08d9  2017-04-24     True          1
2       NaN  air_00a91d42b08b08d9  2017-04-25     True          2
3       NaN  air_00a91d42b08b08d9  2017-04-26     True          3
4       NaN  air_00a91d42b08b08d9  2017-04-27     True          4


##

#### 2.1.5 Merge dataset

- **Input:** ta sẽ có các file train như `air_visit.csv`, `air_store_info.csv` và file test `submission.csv`
- **Output:** ta sẽ nhận được một file gộp lại
- **Mục đích:** để khi đưa vào mô hình thì mô hình sẽ thông qua các thuộc tính để học và dự đoán

In [14]:
data = pd.concat((air_visit, submission))
print(data['is_test'].dtype)
data['is_test']  = data['is_test'].fillna(False).astype('bool') #vì cột khi ép có type là object, dùng astype để chuyển sang boolean
data = pd.merge(left = data, right = date_info, on = 'visit_date', how = 'left')
data = pd.merge(left = data, right = air_store_info, on = 'air_store_id', how = 'left')
data['visitors'] = data['visitors'].astype(float)
data.head()

object


C:\Users\ngdan\AppData\Local\Temp\ipykernel_11764\3566785811.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['is_test']  = data['is_test'].fillna(False).astype('bool') #vì cột khi ép có type là object, dùng astype để chuyển sang boolean


,air_store_id,visit_date,visitors,was_nil,is_test,is_number,day_of_week,is_holiday,prev_holiday,after_holiday,...,air_area_name,latitude,longitude,latitude_str,longitude_str,station_id,station_latitude,station_longitude,station_vincenty,station_great_circle
0,air_00a91d42b08b08d9,2016-07-01,35.0,False,False,NaN,Friday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
1,air_00a91d42b08b08d9,2016-07-02,9.0,False,False,NaN,Saturday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
2,air_00a91d42b08b08d9,2016-07-03,0.0,True,False,NaN,Sunday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
3,air_00a91d42b08b08d9,2016-07-04,20.0,False,False,NaN,Monday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
4,air_00a91d42b08b08d9,2016-07-05,25.0,False,False,NaN,Tuesday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906


## 2.2. Dữ liệu thời tiết

- **Input:** ta sẽ có dữ liệu 1663 trạm đo thời tiết
- **Output:** ta sẽ có file dữ liệu bao gồm tất cả 1663 trạm đo 

In [16]:
import glob
weather_df = []
for path in glob.glob('weather/1-1-16_5-31-17_Weather/weather_data/*.csv'):
    weather = pd.read_csv(path)
    weather['station_id'] = path.split('\\')[-1].rstrip('.csv')
    weather_df.append(weather)


weather = pd.concat(weather_df, axis='rows')
weather.rename(columns={'calendar_date': 'visit_date'}, inplace=True)
weather.head(10)

,visit_date,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,solar_radiation,deepest_snowfall,total_snowfall,avg_wind_speed,avg_vapor_pressure,avg_local_pressure,avg_humidity,avg_sea_pressure,cloud_cover,station_id
0,2016-01-01,6.0,11.0,0.7,0.0,9.1,NaN,NaN,NaN,2.9,NaN,NaN,NaN,NaN,NaN,aichi__ai-xi-kana__isaai
1,2016-01-02,4.7,10.5,0.0,0.0,6.8,NaN,NaN,NaN,1.4,NaN,NaN,NaN,NaN,NaN,aichi__ai-xi-kana__isaai
2,2016-01-03,7.0,13.8,1.9,0.0,8.6,NaN,NaN,NaN,1.4,NaN,NaN,NaN,NaN,NaN,aichi__ai-xi-kana__isaai
3,2016-01-04,8.8,14.7,2.7,0.0,5.3,NaN,NaN,NaN,2.2,NaN,NaN,NaN,NaN,NaN,aichi__ai-xi-kana__isaai
4,2016-01-05,8.9,14.1,3.4,0.0,4.5,NaN,NaN,NaN,2.7,NaN,NaN,NaN,NaN,NaN,aichi__ai-xi-kana__isaai
5,2016-01-06,9.2,12.7,5.3,0.0,2.0,NaN,NaN,NaN,2.5,NaN,NaN,NaN,NaN,NaN,aichi__ai-xi-kana__isaai
6,2016-01-07,8.2,11.6,6.3,0.0,5.7,NaN,NaN,NaN,3.6,NaN,NaN,NaN,NaN,NaN,aichi__ai-xi-kana__isaai
7,2016-01-08,5.9,10.1,1.1,0.0,4.3,NaN,NaN,NaN,1.7,NaN,NaN,NaN,NaN,NaN,aichi__ai-xi-kana__isaai
8,2016-01-09,6.2,9.8,1.9,0.0,9.2,NaN,NaN,NaN,2.9,NaN,NaN,NaN,NaN,NaN,aichi__ai-xi-kana__isaai
9,2016-01-10,5.5,11.1,-0.9,0.0,5.6,NaN,NaN,NaN,2.6,NaN,NaN,NaN,NaN,NaN,aichi__ai-xi-kana__isaai


- **Nhận xét:** có thể thấy nhiều cột mang giá trị NULL, điều này cũng khá dễ hiểu. Bởi vì:
    * Một số trạm chỉ đo những thuộc tính nhất định
    * Một số trạm chỉ đo được thời tiết đặc trưng của nơi đó nên sẽ không có một số thuộc tính khác, ví dụ vùng nắng nóng quanh năm không thể nào đo được thuộc tính tuyết rơi

### 2.2.2 Trích xuất dữ liệu thời tiết

- **Input:** dữ liệu từ bảng weather ta đã tạo trước đó
- **Output:** trích xuất ra những thuộc tính cần thiết để phù hợp với mô hình
- **Ý tưởng:** Vì sao lại có việc trích xuất ở đây?
    * Đó là bởi vì số khách hàng của chúng ta sẽ tính theo ngày nên việc thuộc tính `high_temperature` và `low_temperature` sẽ không giúp ích được gì nhiều, chỉ cần `avg_temperature` là đủ
    * Một số thuộc tính thì lại không cần thiết hoặc do missing quá nhiều nên chúng ta bỏ qua
** Tóm lại chúng ta sẽ chọn 2 thuộc tính: `avg_temperature` và `percitation`**

In [17]:
means = weather.groupby('visit_date')[['avg_temperature', 'precipitation']].mean().reset_index()
means.rename(columns={'avg_temperature': 'global_avg_temperature', 'precipitation': 'global_precipitation'}, inplace=True)
weather = pd.merge(left=weather, right=means, on='visit_date', how='left')
weather['avg_temperature'].fillna(weather['global_avg_temperature'], inplace=True)
weather['precipitation'].fillna(weather['global_precipitation'], inplace=True)

weather[['visit_date', 'avg_temperature', 'precipitation']].head()

C:\Users\ngdan\AppData\Local\Temp\ipykernel_11764\3051333561.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  weather['avg_temperature'].fillna(weather['global_avg_temperature'], inplace=True)
C:\Users\ngdan\AppData\Local\Temp\ipykernel_11764\3051333561.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values 

,visit_date,avg_temperature,precipitation
0,2016-01-01,6.0,0.0
1,2016-01-02,4.7,0.0
2,2016-01-03,7.0,0.0
3,2016-01-04,8.8,0.0
4,2016-01-05,8.9,0.0


In [18]:
data['visit_date'] = pd.to_datetime(data['visit_date'])
data.index = data['visit_date']
# data.sort_values(['air_store_id', 'visit_date'], inplace=True)

data.head()

,air_store_id,visit_date,visitors,was_nil,is_test,is_number,day_of_week,is_holiday,prev_holiday,after_holiday,...,air_area_name,latitude,longitude,latitude_str,longitude_str,station_id,station_latitude,station_longitude,station_vincenty,station_great_circle
visit_date,,,,,,,,,,,,,,,,,,,,,
2016-07-01,air_00a91d42b08b08d9,2016-07-01,35.0,False,False,NaN,Friday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
2016-07-02,air_00a91d42b08b08d9,2016-07-02,9.0,False,False,NaN,Saturday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
2016-07-03,air_00a91d42b08b08d9,2016-07-03,0.0,True,False,NaN,Sunday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
2016-07-04,air_00a91d42b08b08d9,2016-07-04,20.0,False,False,NaN,Monday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906
2016-07-05,air_00a91d42b08b08d9,2016-07-05,25.0,False,False,NaN,Tuesday,0,0.0,0.0,...,Tōkyō-to Chiyoda-ku Kudanminami,35.694003,139.753595,"""35.6940027""","""139.7535951""",tokyo__tokyo-kana__tonokyo,35.691667,139.75,0.416011,0.415906


## 2.3. Tiền xử lí dữ liệu

### 2.2.1 Xử lí giá trị ngoại lai